# PrimeurVision — Évaluation du modèle YOLOv8

Évaluation du meilleur modèle issu de l'entraînement sur le **jeu de test** (36 images, jamais vues pendant l'entraînement).

**Métriques** : mAP@50, mAP@50-95, Précision, Recall, AP par classe

**Classes** : carotte (0), aubergine (1), citron (2), pomme_de_terre (3), radis (4), tomate (5)

## 1. Installation

In [1]:
!pip install ultralytics -q

## 2. Imports

In [2]:
import os
import shutil
import random
import glob
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from ultralytics import YOLO

## 3. Chargement du modèle et du dataset

**Colab** : charge depuis Google Drive.  
**Local** : charge depuis `models/best_yolov8n_primeurvision.pt` dans le projet.

In [ ]:
import sys

ON_COLAB = 'google.colab' in sys.modules or 'google.colab' in str(sys.path)

if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    MODEL_PATH  = '/content/drive/MyDrive/PrimeurVision/models/best_yolov8s_primeurvision_v2.pt'
    DATASET_SRC = '/content/drive/MyDrive/PrimeurVision/dataset'
    MODELS_DIR  = '/content/drive/MyDrive/PrimeurVision/models'
    WORK_DIR    = '/content/dataset'
    if os.path.exists(WORK_DIR):
        shutil.rmtree(WORK_DIR)
    shutil.copytree(DATASET_SRC, WORK_DIR)
else:
    PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
    WORK_DIR     = os.path.join(PROJECT_ROOT, 'dataset')
    MODELS_DIR   = os.path.join(PROJECT_ROOT, 'models')
    MODEL_PATH   = os.path.join(MODELS_DIR, 'best_yolov8s_primeurvision_v2.pt')
    print(f"Mode local — dataset : {WORK_DIR}")

CONF_THRESHOLD = 0.25

# Charger et mettre à jour la config
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

data_config['path']  = WORK_DIR
data_config['train'] = 'images/train'
data_config['val']   = 'images/val'
data_config['test']  = 'images/test'
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

CLASS_NAMES = data_config['names']

# Charger le modèle
model = YOLO(MODEL_PATH)
print(f"Modèle chargé : {MODEL_PATH}")
print(f"Classes : {list(CLASS_NAMES.values())}")
print(f"Images de test : {len(os.listdir(os.path.join(WORK_DIR, 'images', 'test')))}")

## 4. Évaluation quantitative sur le test

In [4]:
metrics = model.val(data=data_yaml_path, split='test', conf=CONF_THRESHOLD)

print("=" * 45)
print("  RÉSULTATS SUR LE JEU DE TEST")
print("=" * 45)
print(f"  mAP@50    : {metrics.box.map50:.4f}")
print(f"  mAP@50-95 : {metrics.box.map:.4f}")
print(f"  Précision : {metrics.box.mp:.4f}")
print(f"  Recall    : {metrics.box.mr:.4f}")
print("-" * 45)
print("  AP@50 par classe :")
for i, name in CLASS_NAMES.items():
    ap50 = metrics.box.ap50[i] if i < len(metrics.box.ap50) else 0
    bar  = '█' * int(ap50 * 20)
    print(f"  {name:20s} : {ap50:.4f}  {bar}")
print("=" * 45)

Ultralytics 8.4.14 🚀 Python-3.11.14 torch-2.10.0 CPU (Apple M3)
Model summary (fused): 73 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1095.8±1329.7 MB/s, size: 1921.0 KB)
val: Scanning /Users/eugenie/Desktop/M2-SISE/13 - Deep learning - Machine learning - Computer Vision/projet_computer_vision/dataset/labels/test... 36 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 36/36 2.0Kit/s 0.0s
val: New cache created: /Users/eugenie/Desktop/M2-SISE/13 - Deep learning - Machine learning - Computer Vision/projet_computer_vision/dataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.4s/it 4.3s3.8ss
                   all         36        482      0.502      0.431      0.455      0.311
               carotte          8        232      0.457      0.228       0.33      0.184
             aubergine          7         30      0.643        0.3      0.482  

## 5. Matrice de confusion et courbes

In [5]:
eval_dir = str(metrics.save_dir)

# Matrice de confusion normalisée
for fname in ['confusion_matrix_normalized.png', 'confusion_matrix.png']:
    confusion_img = os.path.join(eval_dir, fname)
    if os.path.exists(confusion_img):
        plt.figure(figsize=(8, 8))
        plt.imshow(Image.open(confusion_img))
        plt.axis('off')
        plt.title('Matrice de confusion — jeu de test')
        plt.show()
        break

# Courbe Précision-Rappel
pr_img = os.path.join(eval_dir, 'PR_curve.png')
if os.path.exists(pr_img):
    plt.figure(figsize=(10, 6))
    plt.imshow(Image.open(pr_img))
    plt.axis('off')
    plt.title('Courbe Précision-Rappel — jeu de test')
    plt.show()

# Courbes d'entraînement (depuis dossier models/)
curves_img = os.path.join(MODELS_DIR, 'results.png')
if os.path.exists(curves_img):
    plt.figure(figsize=(18, 8))
    plt.imshow(Image.open(curves_img))
    plt.axis('off')
    plt.title("Courbes d'entraînement (phase 2)")
    plt.show()
else:
    print(f"(courbes d'entraînement non trouvées dans {MODELS_DIR})")

<Figure size 800x800 with 1 Axes>

<Figure size 1800x800 with 1 Axes>

## 6. Résultats qualitatifs — Prédictions réussies

Exemples de détections correctes sur le jeu de test.

In [6]:
test_images = glob.glob(os.path.join(WORK_DIR, 'images', 'test', '*.jpg'))
sample_test = random.sample(test_images, min(6, len(test_images)))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
for ax, img_path in zip(axes.flatten(), sample_test):
    result = model.predict(img_path, conf=CONF_THRESHOLD, verbose=False)[0]
    ax.imshow(result.plot()[:, :, ::-1])
    n_det = len(result.boxes)
    ax.set_title(
        f"{os.path.basename(img_path)[:25]}\n({n_det} détection(s))",
        fontsize=8
    )
    ax.axis('off')

plt.suptitle('Exemples de prédictions réussies — jeu de test', fontsize=14)
plt.tight_layout()
plt.show()

<Figure size 1800x1200 with 6 Axes>

## 7. Analyse des erreurs — Cas difficiles

On identifie les images où le modèle détecte peu ou avec une faible confiance. Ces cas révèlent les limites du modèle : objets partiellement visibles, occlusions, angles atypiques, ou classes sous-représentées dans le dataset.

In [7]:
# Collecter toutes les prédictions sur le jeu de test
all_results = []
for img_path in test_images:
    result   = model.predict(img_path, conf=CONF_THRESHOLD, verbose=False)[0]
    n_det    = len(result.boxes)
    max_conf = float(result.boxes.conf.max()) if n_det > 0 else 0.0
    all_results.append((img_path, result, n_det, max_conf))

# Trier par confiance maximale croissante (les cas les plus difficiles en premier)
all_results.sort(key=lambda x: x[3])

# Afficher les 6 images les plus problématiques
worst = all_results[:min(6, len(all_results))]
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
for ax, (img_path, result, n_det, max_conf) in zip(axes.flatten(), worst):
    ax.imshow(result.plot()[:, :, ::-1])
    ax.set_title(
        f"{os.path.basename(img_path)[:25]}\n"
        f"{n_det} det. | conf max = {max_conf:.2f}",
        fontsize=8, color='crimson'
    )
    ax.axis('off')

plt.suptitle('Cas difficiles — confiance maximale la plus faible', fontsize=14)
plt.tight_layout()
plt.show()

# Résumé
n_zero = sum(1 for _, _, n, _ in all_results if n == 0)
confs  = [c for _, _, n, c in all_results if n > 0]
print(f"Images sans aucune détection : {n_zero}/{len(all_results)}")
if confs:
    print(f"Confiance moyenne (images avec détections) : {np.mean(confs):.3f}")
    print(f"Confiance médiane : {np.median(confs):.3f}")

<Figure size 1800x1200 with 6 Axes>

Images sans aucune détection : 1/36
Confiance moyenne (images avec détections) : 0.769
Confiance médiane : 0.820


## 8. Distribution des confiances et des classes détectées

In [8]:
COLORS = ['#FF8C00', '#9B59B6', '#FFD700', '#8B4513', '#E74C3C', '#FF4444']

all_confs    = []
class_counts = {v: 0 for v in CLASS_NAMES.values()}

for img_path in test_images:
    result = model.predict(img_path, conf=CONF_THRESHOLD, verbose=False)[0]
    for box in result.boxes:
        all_confs.append(float(box.conf))
        name = CLASS_NAMES.get(int(box.cls), f'class_{int(box.cls)}')
        class_counts[name] = class_counts.get(name, 0) + 1

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Distribution des scores de confiance
ax1.hist(all_confs, bins=20, color='steelblue', edgecolor='white')
ax1.axvline(CONF_THRESHOLD, color='red', linestyle='--', label=f'Seuil = {CONF_THRESHOLD}')
ax1.set_title('Distribution des scores de confiance (test)')
ax1.set_xlabel('Confiance')
ax1.set_ylabel('Nombre de détections')
ax1.legend()

# Nombre de détections par classe prédite
bars = ax2.bar(class_counts.keys(), class_counts.values(), color=COLORS)
ax2.bar_label(bars)
ax2.set_title('Détections par classe (jeu de test)')
ax2.set_xlabel('Classe')
ax2.set_ylabel('Nb détections')
ax2.tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

print(f"Total détections : {len(all_confs)}")
if all_confs:
    print(f"Confiance moyenne : {np.mean(all_confs):.3f}")
    print(f"Confiance médiane : {np.median(all_confs):.3f}")

<Figure size 1400x500 with 2 Axes>

Total détections : 361
Confiance moyenne : 0.522
Confiance médiane : 0.482
